# Eliminação de Gauss

### Prof. Pedro Peixoto <ppeixoto@usp.br>

Abril 2022


In [1]:
import numpy as np
import sys
import time

### O problema Ax=b

In [2]:
# Tamanho do sistema
n = 300

def create_matrix(n):
    # Matriz aumentada A|b
    a = np.zeros((n,n+1))

    # Matriz A
    for i in range(n):
        for j in range(n):
            if (i == j) :
                a[i,j] = 2
            elif (abs(i-j)< 2):
                a[i,j] = -1

    # Vetor b 
    for i in range(n):
        a[i,n] = 1

    #print(a)
    return a
    

### O esquema clássico, sem pivotamento

In [3]:
a = create_matrix(n)
x = np.zeros(n)
A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

start_time = time.time()
# Escalonamento
for i in range(n):
    if a[i,i] == 0.0:
        sys.exit('Ainda não implementei pivotamento :-( ')
        
    for j in range(i+1, n):
        ratio = a[j,i]/a[i,i]
        for k in range(n+1):
            a[j,k] = a[j,k] - ratio * a[i,k]

# Substituição
x[n-1] = a[n-1,n]/a[n-1,n-1]

for i in range(n-2,-1,-1):
    x[i] = a[i,n]
    
    for j in range(i+1,n):
        x[i] = x[i] - a[i,j]*x[j]
    
    x[i] = x[i]/a[i,i]
tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(b-A@x)))


Soma da Solução: 2272550.0000000903  Tempo que levou:  6.581117868423462

Solução: 
150.00000000000156

Teste: 
1.8189894035458565e-12


### Dá para vetorizar?

In [4]:
a = create_matrix(n)
x = np.zeros(n)
A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

start_time = time.time()

# Escalonamento
for i in range(n):
    if a[i,i] == 0.0:
        sys.exit('Ainda não implementei pivotamento :-( ')
        
    for j in range(i+1, n):        
        #Vetorizei aqui!
        a[j,:] = a[j,:] - a[j,i]/a[i,i] * a[i,:]

# Substituição
x[n-1] = a[n-1,n]/a[n-1,n-1]

for i in range(n-2,-1,-1):
    #Vetorizei aqui!
    x[i] = (a[i,n] - np.dot(a[i,i+1:n], x[i+1:n]))/a[i,i]

tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(A@x- b)))

Soma da Solução: 2272550.0000000903  Tempo que levou:  0.17776012420654297

Solução: 
150.00000000000156

Teste: 
1.8189894035458565e-12


### Dá para vetorizar mais?

Para isso tem que mudar o algoritmos, vejamos um algoritmo de decomposição LU (que vai serir de base para Elim. Gaussiana vetorial): Algoritmo de Crout (ou Doolittle, dependendo de onde colocamos a diagonal unitária).

O problema agora é Ax=LUx=b


In [5]:
a = create_matrix(n)

A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

start_time = time.time()

#Guardo tanto L quanto U em uma única matriz!!!!
LU = np.eye(n)
# @ faz multiplicação de matrizes usando numpy
for i in range(n):
    #Varre linhas superiores (Upper)
    LU[i,i:] = A[i,i:]-LU[i,:i] @ LU[:i,i:]
    #Varre colunas inferiores (Lower)
    LU[(i+1):,i] = ( A[(i+1):,i]- LU[(i+1):,:i] @ LU[:i,i] ) / LU[i,i]


# Substituição
# LUx=b
# Ly=b
y = np.zeros(n)
# Ly=b
y[0] = b[0]
for i in range(1,n,1):
    #Vetorizei aqui!
    y[i] = (b[i] - np.dot(LU[i,:i], y[:i]))

# Ux=y
x = np.zeros(n)
x[n-1] = y[n-1]/LU[n-1,n-1]
for i in range(n-2,-1,-1):
    #Vetorizei aqui!
    x[i] = (y[i] - np.dot(LU[i,i+1:], x[i+1:]))/LU[i,i]

tempo = time.time() - start_time

print("Soma da Solução:", np.sum(x), " Tempo que levou: ", tempo)

# Solução
print('\nSolução: ')
print(x[0])
print('\nTeste: ')
print(np.max(np.abs(A@x- b)))

#Testes
U = np.triu(LU)
L = np.tril(LU)
np.fill_diagonal(L, 1.0)
print('\nTeste LU: ')
print(np.max(np.max(np.abs(L@U-A))))

Soma da Solução: 2272550.0000000903  Tempo que levou:  0.005848884582519531

Solução: 
150.00000000000156

Teste: 
1.8189894035458565e-12

Teste LU: 
1.1102230246251565e-16


#### Desafio: Você consegue usar essa ideia do LU para fazer diretamente a solução do sistema linear vetorial, com apenas um loop, sem precisar calcular L e U explicitamente como fizemos?

## E a precisão das contas?

Vamos simular o algoritmo como se ele estivesse rodando em um computador com menos bits, arredondando as contas para um número menos de algarismos significativos.

In [6]:
from sigfig import round

sigfig = 3

In [7]:
def gauss_elim_precisao_reduzida(a, n, sigfig):
    
    # Escalonamento
    for i in range(n):
        if a[i,i] == 0.0:
            sys.exit('Ainda não implementei pivotamento :-( ')
            
        for j in range(i+1, n):
            
            ratio = round(a[j,i]/a[i,i], sigfigs = sigfig)
            for k in range(n+1):
                a[j,k] = round(a[j,k] - round(ratio * a[i,k], sigfigs = sigfig), sigfigs = sigfig)


    # Substituição
    x[n-1] = round(a[n-1,n]/a[n-1,n-1], sigfigs = sigfig)

    for i in range(n-2,-1,-1):
        x[i] = a[i,n]
        
        for j in range(i+1,n):
            x[i] = round(x[i] - round(a[i,j]*x[j], sigfigs = sigfig), sigfigs = sigfig)
        
        x[i] = round(x[i]/a[i,i], sigfigs = sigfig)

    return x

In [8]:

a = np.matrix('1 4 52 57; 27 110 -3 134; 22 2 14 38')
n = 3

x = np.zeros(n)
A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

x = gauss_elim_precisao_reduzida(a, n, sigfig)

# Solução
print('\nSolução: ')
print(x)
print('\nTeste: ')
print(b-A@x)



Solução: 
[4.5  0.   1.01]

Teste: 
[-2.000e-02  1.553e+01 -7.514e+01]


/home/pedrosp/anaconda3/lib/python3.8/site-packages/sigfig/sigfig.py:585: UserWarning: warning: 3 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))
/home/pedrosp/anaconda3/lib/python3.8/site-packages/sigfig/sigfig.py:585: UserWarning: warning: 3 significant figures requested from number with only 2 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


O erro foi muito grande!!!!

Trocando a ordem das equações:

In [9]:
a = np.matrix(' 27 110 -3 134; 22 2 14 38; 1 4 52 57')
n = 3

x = np.zeros(n)
A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

x = gauss_elim_precisao_reduzida(a, n, sigfig)


# Solução
print('\nSolução: ')
print(x)
print('\nTeste: ')
print(b-A@x)


Solução: 
[1. 1. 1.]

Teste: 
[0. 0. 0.]


Percebem a diferença de precisão? (é o mesmo computador, só mudamos a ordem das contas...)

Vamos colocar piovtamento parcial agora:

In [10]:
def gauss_elim_pivot(a, n, sigfig = 20):
    
    # Escalonamento
    for i in range(n):
        #Pivotar linha i com linha do máximo valor em módulo!
        kmax = i
        for k in range(i+1,n):
            if np.abs(a[k,kmax]) > np.abs(a[kmax,i]):
                kmax = k

        #Troca as linhas  
        if (kmax != i):
            print("Troquei linha ", i, " pela linha ", kmax)
            a[[i, kmax], :] = a[[kmax, i], :]

        if a[i,i] == 0.0:
            sys.exit('Sistema singular!!')
            
        for j in range(i+1, n):
            
            ratio = round(a[j,i]/a[i,i], sigfigs = sigfig)
            for k in range(n+1):
                a[j,k] = round(a[j,k] - round(ratio * a[i,k], sigfigs = sigfig), sigfigs = sigfig)


    # Substituição
    x[n-1] = round(a[n-1,n]/a[n-1,n-1], sigfigs = sigfig)

    for i in range(n-2,-1,-1):
        x[i] = a[i,n]
        
        for j in range(i+1,n):
            x[i] = round(x[i] - round(a[i,j]*x[j], sigfigs = sigfig), sigfigs = sigfig)
        
        x[i] = round(x[i]/a[i,i], sigfigs = sigfig)

    return x

In [11]:

a = np.matrix('1 4 52 57; 27 110 -3 134; 22 2 14 38')
n = 3

x = np.zeros(n)
A = np.copy(a[:,:-1])
b = np.copy(a[:,n]).reshape((n))

x = gauss_elim_pivot(a, n, sigfig)

# Solução
print('\nSolução: ')
print(x)
print('\nTeste: ')
print(b-A@x)


Troquei linha  0  pela linha  1
Troquei linha  1  pela linha  2

Solução: 
[1. 1. 1.]

Teste: 
[0. 0. 0.]
